# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [78]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [79]:
#corrected-data_path = 'weatherAPI_data.csv'
corrected_data_path = "../output_data/cities.csv"

corrected_data_df = pd.read_csv(corrected_data_path)
corrected_data_df.head(20)


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,3833367,Ushuaia,20,AR,1603764187,39,-54.80,-68.30,48.20,18.34
1,4030556,Rikitea,100,PF,1603764188,86,-23.12,-134.97,75.38,14.34
2,3939761,Hualmay,79,PE,1603764189,82,-11.10,-77.61,63.93,8.14
3,556268,Ostrovnoy,100,RU,1603764164,94,68.05,39.51,41.43,23.94
4,4231997,Avera,40,US,1603764191,90,33.19,-82.53,64.40,3.85
5,5106841,Albany,100,US,1603763992,89,42.60,-73.97,51.01,1.50
6,2136150,Luganville,40,VU,1603764193,66,-15.53,167.17,84.20,11.41
7,2155415,New Norfolk,88,AU,1603764195,48,-42.78,147.06,59.00,3.00
8,3421319,Nuuk,93,GL,1603764196,63,64.18,-51.72,28.40,4.70
9,3601282,Sinuapa,88,HN,1603764197,90,14.45,-89.18,66.67,2.13


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [80]:
gmaps.configure(api_key=g_key)
locations = corrected_data_df[["Lat","Lng"]]
humidity = corrected_data_df["Humidity"]

In [81]:
fig = gmaps.figure(center=(50.0,-5.0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [85]:
#make a new df with cities with 0 cloudiness, wind speed <10 and humidity<20
new_df = corrected_data_df[(corrected_data_df["Cloudiness"] == 0) & (corrected_data_df["Wind Speed"] <= 10) & (corrected_data_df["Humidity"] <= 20)].dropna()
new_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
192,2444995,Filingué,0,NE,1603764409,16,14.35,3.32,80.49,1.63
395,2245170,Podor,0,MR,1603764648,12,16.65,-14.96,88.77,8.75
409,2077895,Alice Springs,0,AU,1603764420,17,-23.70,133.88,77.00,8.05
416,2447513,Arlit,0,NE,1603764672,19,18.74,7.39,73.81,6.91
533,102527,Sakakah,0,SA,1603764805,11,29.97,40.21,69.80,5.82
534,2377450,Nouakchott,0,MR,1603764806,20,18.09,-15.98,77.00,3.36
536,2034754,Shunyi,0,CN,1603764808,17,40.12,116.65,57.99,8.95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [86]:
#adda column name Hotel_Name
hotel_df = new_df.loc[:,["City_ID", "City", "Country", "Lat", "Lng"]]
hotel_df
hotel_df["Hotel_Name"]=""
hotel_df

,City_ID,City,Country,Lat,Lng,Hotel_Name
192,2444995,Filingué,NE,14.35,3.32,
395,2245170,Podor,MR,16.65,-14.96,
409,2077895,Alice Springs,AU,-23.70,133.88,
416,2447513,Arlit,NE,18.74,7.39,
533,102527,Sakakah,SA,29.97,40.21,
534,2377450,Nouakchott,MR,18.09,-15.98,
536,2034754,Shunyi,CN,40.12,116.65,


In [87]:
# find the closest hotel to the narrowed down cities

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000 , 
    "keyword": "hotel",
    "types": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    time.sleep(1)    
    # get hotel location
    lat = row['Lat']
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params['location'] = f"{lat},{lng}"

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    #print(response)
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel_Name'] = results[0]['name']
       # print(results[0]['name'])
        #types_df.loc[index, 'address'] = results[0]['vicinity']
        #types_df.loc[index, 'price_level'] = results[0]['price_level']
        #types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Closest hotel in Filingué is Hôtel CTK.
------------
Closest hotel in Podor is La cour du fleuve.
------------
Closest hotel in Alice Springs is DoubleTree by Hilton Hotel Alice Springs.
------------
Closest hotel in Arlit is Hôtel Telwa Bungalow.
------------
Closest hotel in Sakakah is Alnuzl Special Hotel.
------------
Closest hotel in Nouakchott is AZALAÏ HOTEL NOUAKCHOTT (MARHABA ).
------------
Closest hotel in Shunyi is Shunyi Hotel （South Gate）.
------------


In [88]:
hotel_df

,City_ID,City,Country,Lat,Lng,Hotel_Name
192,2444995,Filingué,NE,14.35,3.32,Hôtel CTK
395,2245170,Podor,MR,16.65,-14.96,La cour du fleuve
409,2077895,Alice Springs,AU,-23.70,133.88,DoubleTree by Hilton Hotel Alice Springs
416,2447513,Arlit,NE,18.74,7.39,Hôtel Telwa Bungalow
533,102527,Sakakah,SA,29.97,40.21,Alnuzl Special Hotel
534,2377450,Nouakchott,MR,18.09,-15.98,AZALAÏ HOTEL NOUAKCHOTT (MARHABA )
536,2034754,Shunyi,CN,40.12,116.65,Shunyi Hotel （South Gate）


In [89]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [90]:
# Add marker layer ontop of heat map
markerlayer=gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markerlayer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))